In [1]:
### Autoren: Omar De-Mitri, Hang Beom Kim, Martina Köhler, Januray 2022

# Übung Sicherheitskritische Anwendungen

## Einleitung

In diesem Modul soll mithilfe eines neuronalen Netzes, welches auf Bildern trainiert wurde, überprüft werden, ob die Anwendung sicher ist oder nicht. Dafür wurde eine Kamera über dem Arbeitsplatz angebracht. Der Datensatz besteht aus Bildern von dieser Kamera. Wenn keine Hand auf den Bildern zu sehen ist, dann ist der Arbeitsplatz sicher. Sobald eine Hand zu sehen ist, soll das neuronale Netz den Arbeitsplatz als unsicher kategorisieren. 

Diese Art von Klassifizierung kann man sich bei der Ein- und Abschaltung eines Greifroboters vorstellen, der sich, sobald eine menschliche Hand in die Nähe kommt, sofort ausschalten muss. 

## Schritt 1: Herunterladen der Daten

### Aufgabe 1.1

Als erstes muss der Datensatz heruntergeladen werden. 
Innerhalb dieses Ordners befindet sich die Datei "AKIPRO_safety_application_dataset.zip". Dafür entpacken Sie bitte diesen Ordner "AKIRPO_safetey_application_dataset" in dem selben Ordner, in dem auch das Jupyter Notebook `AKIPRO_safety_applications.ipynb` liegt. 

Die ursprüngliche Ordnerstruktur von "safe" und "unsafe" muss beibehalten werden, damit das Skript die Bilder korrekt klassifizieren kann. Diese Struktur wird später genutzt, um die Daten korrekt zu kategorisieren.


## Schritt 2: Exploration der Daten

Relevante Bibliotheken in das Skript reinladen:

In [1]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt



Zunächst geben wir dem Skript mithilfe von `os.path` die korrekte Ordnerstruktur. Falls Ihr Ordner, der die Daten beinhaltet, doch einen anderen Namen trägt, kann das hier angepasst werden. 

In [4]:
dir_str = "AKIPRO_safety_application_dataset"
safe_str = "safe"
unsafe_str = "unsafe"

# os.path.join verbindet Strings zu Dateipfaden und funktioniert auf allen Betriebssystemem
safe_dir = os.path.join(dir_str, safe_str)
unsafe_dir = os.path.join(dir_str, unsafe_str)


### Aufgabe 2.1 

Das neuronale Netz wird mit sowohl Bildern, die als sicher kategorisiert wurden, als auch mit Bildern, die als unsicher klassifiziert wurden trainiert. Im Ordner des Datensatzes sind die Bilder kategorisiert, indem man sie in einem der Ordner `safe` oder `unsafe` abgelegt wurden. Zunächst ist es wichtig, erste Einblicke in die Daten zu bekommen, mit denen das neuronale Netz trainiert wird.

Dafür schauen Sie sich bitte ein paar Bilder aus beiden Kategorien an. Somit erhalten Sie ein erstes Gefühl für die Art von Daten, die das neuronale Netz als Input bekommt. 

Welche Größen haben die einzelnen Bildarrays?

In [ ]:
# Bild der sicheren Kategorie laden
img_path = os.path.join(safe_dir, "akipro_0000000000.jpg")
img = cv2.imread(img_path)

# Bild der sicheren Kategorie anzeigen
plt.imshow(img[:, :, ::-1])
plt.xticks([]), plt.yticks([])  # Dadurch werden keine Striche an den Achsen angezeigt
plt.title("sicher")
plt.show()

# Bild der unsicheren Kategorie laden

# Bild der unsicheren Kategorie anzeigen

# Maximal- und Minimalwerte des Bildes
print(f"Maximum der Bildwerte: {np.max(img)}, , Minimum der Bildwerte: {np.min(img)}")

# Größe des Bildes
# print(f"Größe des Bildes: {}")

In [ ]:
%run Tipps/Tipp_2_1.py

In [ ]:
%load Loesung/Loesung_2_1.py

## Schritt 3: Datensatz mit Keras erstellen

Nun können die Daten mithilfe von Keras in das Skript geladen werden. Dafür werden in den Folgenden Zellen verschiedenen Funktionen erstellt und verwendet. Versuchen Sie die einzelnen Schritte nachzuvollziehen und ergänzen Sie den Code an den angegebenen Stellen. Die nachfolgende Zelle importiert die notwendigen Bibliotheken und Funktionen. 

In [7]:
# all imports
import random
import pandas as pd
import csv

# ML libraries
import tensorflow as tf
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint

### Aufgabe 3.1

In diesem Abschnitt werden die .csv Dateien für den Trainings- und Testdatensatz geladen. Bitte ergänzen Sie den Code an den zwei angegebenen Stellen. 

In [ ]:
def create_csv_file(dir):
    # Zunächst werden die Pfade definiert. Bitte ergänzen Sie die Zeile für das Test.csv
    csv_path_train = os.path.join(dir, "Training.csv")
    csv_path_test = os.path.join(dir, "Test.csv")

    # Jetzt werden die Dateien erstellt und die Spaltennamen geschrieben
    # Training
    with open(csv_path_train, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ImageId", "PredictionString"])
    # Test
    # Platz für Ihren Code

    # Anzeigen der verschiedenen Klassen -> Es werden nur die Ordner verwendet
    list_of_classes = [element for element in os.listdir(dir) if not os.path.isfile(os.path.join(dir, element))]
    print(f"Liste aller Klassen: {str(list_of_classes)}")

    # Iteriere über alle Klassen
    for current_class in list_of_classes:
        class_dir = os.path.join(dir, current_class)
        # Liste aller .jpg Dateien im Ordner der aktuellen Klasse
        #for file in os.listdir(class_dir)
        list_of_files = [file for file in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, file)) and file.endswith(".jpg")]
        
        # Dateien zufällig mischen
        random.seed(0)
        random.shuffle(list_of_files)

        # Dateien in Trainings- und Testdaten splitten
        train_end = int(len(list_of_files) * 0.75)
        train_files = list_of_files[:train_end]
        test_files = list_of_files[train_end:]

        # Hier werden die Dateiname in die .csv Dateien geschrieben
        # Training
        with open(csv_path_train, 'a', newline='') as file:
            writer = csv.writer(file)
            for file in train_files:
                writer.writerow([file, current_class])

        # Test
        # Platz für Ihren Code


# Verzeichnis, in dem der Datensatz liegt
input_dir = 'AKIPRO_safety_application_dataset'

# Erstellen der .csv Dateien
create_csv_file(input_dir)

In [ ]:
%load Loesung/Loesung_3_1.py

### Aufgabe 3.2 
Im folgenden Codeabschnitt soll eine Funktion erstellt werden, mit der die Dateien gemäß der .csv Dateiein geladen werden können.
Beantworten Sie die Fragen im Code. 
Die [Pandas API Reference](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) kann Ihnen dabei helfen.

In [ ]:
def load_dataset(path, csv_file):
    # Diese Zeile lädt die eben erstelle .csv Datei in ein Pandas DataFrame
    csv_df = pd.read_csv(os.path.join(path, csv_file), dtype={'ImageId':str,'PredictionString':str})
    # Gibt den "Kopf" des DataFrames aus
    print(csv_df.head())
    data = []
    csv_df['ImageId'] = csv_df['ImageId']
    
    for idx, data_row in csv_df.iterrows():
        image_id = data_row["ImageId"]
        label = data_row["PredictionString"]
        data_tuple = (os.path.join(path, label, image_id), label)
        data.append(list(data_tuple))

    df = pd.DataFrame(data, columns = ['imagename', 'label'])

    # Diese Zeile mischt die Zeilen des DataFrames. Warum benötigen Sie diese Zeile? Was gibt der Parameter frac an, warum ist er hier 1?
    df = df.sample(frac=1).reset_index(drop=True)
    return df

# Laden der Datensätze
df_train = load_dataset(input_dir,'Training.csv')
df_test = load_dataset(input_dir,'Test.csv')

In [ ]:
%run Loesung/Loesung_3_2.py

### Aufgabe 3.3 
Preprocessing. Versuchen Sie die nachfolgende Funktion nachzuvollziehen. Warum wird dieser Schritt benötigt?

In [14]:
def preprocessing(image):
    image = np.array(image)
    image /= 255

    return image

In [ ]:
%run Loesung/Loesung_3_3.py

### Aufgabe 3.4
Im Folgenden Abschnitt werden sogenannte Generatoren für die Datensätze erstellt. Diese Schritte sind etwas komplexer. Daher beschränkt sich Ihre Aufgabe darauf den folgenden Code möglichst gut zu verstehen. Lesen Sie sich dazu die [Dokumentation des ImageDataGenerators](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) durch.

In [ ]:
# Breite und Höhe der Bilder
width, height = 720, 540

def create_generator(dataset, df):
    # Hinweis: Nur die Trainingsdaten werden augmentiert. Rotation, horizontale/vertikale Spiegelung, etc.
    trainDataGenerator=ImageDataGenerator(
            rescale = None,
            rotation_range = 40,
            horizontal_flip = True,
            vertical_flip = True,
            validation_split=0.25,
            brightness_range=[0.3,1.0],
            preprocessing_function=preprocessing
        )
        
    testDataGenerator=ImageDataGenerator(rescale=None, preprocessing_function=preprocessing)

    if dataset == 'Training':
        # Training
        train_generator=trainDataGenerator.flow_from_dataframe(
            dataframe=df,
            directory=None,
            x_col="imagename",
            y_col="label",
            subset="training",
            batch_size=32,
            shuffle=True,
            seed=42,
            class_mode="categorical",
            target_size=(width, height))

        # Validierung
        valid_generator=trainDataGenerator.flow_from_dataframe(
            dataframe=df,
            directory=None,
            x_col="imagename",
            y_col="label",
            subset="validation",
            batch_size=32,
            shuffle=True,
            seed=42,
            class_mode="categorical",
            target_size=(width, height))

        return train_generator, valid_generator
    
    elif dataset == 'Test':
        test_generator=testDataGenerator.flow_from_dataframe(
            dataframe=df,
            directory=None,
            x_col="imagename",
            y_col="label",
            batch_size=32,
            shuffle=True,
            seed=42,
            class_mode="categorical",
            target_size=(width, height))
        
        return test_generator
    
    else:
        AttributeError("Please choose either Training or Test as dataset")


# Erstellen der Generatoren
train_generator, valid_generator = create_generator('Training', df_train)
test_generator = create_generator('Test', df_test)

### Aufgabe 3.5
Wie sind die Labels der Daten im Code repräsentiert?
Schauen Sie sich das Pandas DataFrame `df_train` und den `train_generator` genauer an.
Welche Klasse wird mit 0 und welche Klasse wird mit 1 repräsentiert?

In [65]:
# Platz für Ihren Code

In [ ]:
%run Tipps/Tipp_3_5.py

In [ ]:
%load Loesung/Loesung_3_5.py

### Aufgabe 3.6
Wie viele Bilder gibt es in den jeweiligen Kategorien? (Trainingsdatensatz)

In [ ]:
safe_count, unsafe_count = 0, 0

for idx, data_row in df_train.iterrows():
    # label = ...
    # Ergänzen Sie hier Ihren Code

print(f"Die sichere Klasse beinhaltet {safe_count} Bilder.")
print(f"Die unsichere Klasse beinhaltet {unsafe_count} Bilder.")


In [ ]:
%load Loesung/Loesung_3_6.py

### Aufgabe 3.7 
Interpretieren Sie das Ergebnis aus Aufgabe 3.6. Ist dieser Zustand wünschenswert? Was wären mögliche Lösungsmöglichkeiten?

In [ ]:
%run Loesung/Loesung_3_7.py

## Schritt 4: Modeldefinition & Training

Im nächsten Schritt wird das Modell definiert. Versuchen Sie den Code so gut wie möglich zu verstehen. Die [TensorFlow Reference API](https://www.tensorflow.org/api_docs/python/tf) kann Ihnen dabei helfen.

In [18]:
# create untrained keras model:
def init_model():

    model = keras.Sequential()
    # Eingang des neuronalen Netzes (Dimension der Bilder)
    model.add(tf.keras.Input(shape=(width, height, 3)))
    # Definition der Hidden Layers
    model.add(tf.keras.layers.Conv2D(32, 5, strides=2, activation="relu"))
    model.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
    model.add(tf.keras.layers.Conv2D(32, 4, activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(3))   
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(32))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    # Dieses Layer projeziert den mehrdimensionalen Tensor auf eine Dimension
    model.add(tf.keras.layers.Flatten())
    # Ausgangsschicht
    model.add(tf.keras.layers.Dense(2, activation="softmax"))
    # Definition der Loss Funktion, des Optimierers, der Learning Rate und der Evaluationsmetrik (hier Klassifikationsgenauigkeit)
    model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001), metrics=["acc"])

    return model

def training_phase(model, train_generator):
    # Definition und ggf. Erstellen des Ordners, in dem das Modell gespeichert werden soll
    model_path = 'saved_models'
    model_name = 'lr0-00001_model_akipro_security_training'
    checkpoint_name = os.path.join(model_path, model_name + ".ckpt")
    if not os.path.isdir(model_path):
        os.makedirs(model_path)

    # Definition callbacks. Diese Funktionen werden zyklisch von der model.fit() Funktion aufgerufen
    # EarlyStopping verhindert Overfitting
    # ModelCheckpoint speichert die aktuellen Parameter des Modells in sogenannten Checkpoints
    callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=4), 
                 ModelCheckpoint(checkpoint_name,  verbose=0, save_weights_only=True, save_freq=1)]

    step_size_train=train_generator.n//train_generator.batch_size
    step_size_valid=valid_generator.n//valid_generator.batch_size

    print('start training')
    history = model.fit(
        train_generator,
        steps_per_epoch=step_size_train,
        epochs=50,
        callbacks=callbacks,
        validation_data=valid_generator,
        validation_steps=6,
        verbose=1)

    # Trainingshistorie speichern
    with open(checkpoint_name + '_lr0-00001_history.txt', 'w') as f:
        f.write(str(history.history))
        f.close()
    
    # Model speichern
    model_path = os.path.join(model_path, 'lr0-00001_model_akipro_security_training')
    model.save(model_path)
    return model

### Aufgabe 4.1 

Wie viele Schichten hat das neuronale Netz? Um welche Art von neuronalem Netz handelt es sich? Wie viele trainierbare Parameter hat das Netz?

Platz für Notizen

In [75]:
# Platz für Ihren Code

In [ ]:
%run Tipps/Tipp_4_1.py

In [ ]:
%run Loesung/Loesung_4_1.py

## Trainieren

Das Modell ist bereit trainiert zu werden. Bevor Sie den Code ausführen, beantworten Sie bitte folgende Fragen:

### Aufgabe 4.2
Welche Funktion müssen Sie aufrufen, damit dieses Modell nachtrainiert wird? 



Platz für Notizen

In [ ]:
%run Loesung/Loesung_4_2.py

### Aufgabe 4.3

An welcher Stelle kann die Anzahl der Epochen für das Training geändert werden? Passen Sie diese Hyperparameter an, wenn das Training sehr langsam auf Ihrem Rechner sein sollte. Um das Training zu beschleunigen, können Sie auch `tensorflow-gpu` installieren, wie in Loesung/Loesung_4_2.py bereits erläutert wurde. 

In [ ]:
%run Loesung/Loesung_4_3.py

 Hier können Sie sich entscheiden, ob Sie das vortrainierte Netz nehmen und damit die Übung beenden, oder ob Sie ein Training starten. 

 Die direkt folgende Zelle trainiert ein neues Modell. Die Zelle darunter verwendet das bereits vortrainierte Netz. Probieren Sie ruhig aus, wie sich das Training 2-5 Epochen verhält, auch wenn Sie später das vortrainierte Netz verwenden sollten. 

In [ ]:
tf.keras.backend.clear_session()
# trainiere 
model = init_model()

model = training_phase(model, train_generator)


### Verwenden des vortrainierten Netzes
Falls Sie nicht selbst trainieren möchten oder können, können Sie hier auch das vortrainierte Netz laden.


In [21]:
tf.keras.backend.clear_session()
model = init_model()
model_dir = "saved_models"
model_name = os.path.join(model_dir, "lr-0-0001_akipro.security.model")
model = load_model(model_name)

## Schritt 5: Loss- & Accuracy-Kurve untersuchen
### Aufgabe 5.1

Versuchen Sie nun die Kurve der Lossfunktion und die Kurve der Accuracywerte mithilfe der `matplotlib`-Bibliothek zu erstellen. Die [Dokumentation der plot-Funktion](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html) kann Ihnen dabei helfen. Achten Sie darauf, dass die Plots korrekte Achsenbeschriftungen und eine Legende haben.

In [ ]:
import ast

history_path = os.path.join(model_dir, "model_akipro_security.ckpt_lr-0-0001_history.txt")
with open(history_path, 'r') as file:
    history = file.readlines()

# Kovertieren als Dictionary
hist_dict = ast.literal_eval(history[0])

# Laden in separate Listen
train_loss = hist_dict["loss"]
val_loss = hist_dict["val_loss"]
# Ergänzen Sie hier die zwei Zeilen für die Accuracy

# Loss-Kurve
plt.plot(train_loss, label="train_loss")
# Platz für Ihren Code
plt.show()

# Accuracy-Kurve
# Platz für Ihren Code

In [ ]:
%run Tipps/Tipp_5_1.py

In [ ]:
%load Loesung/Loesung_5_1.py

### Aufgabe 5.2

Wie ist die Kurve der Lossfunktion und die Accuracywerte zu bewerten? Was bedeuten sie? Was wären Accuracywerte überhalb der Zufallsschwelle bei einer binären Klasse?

In [ ]:
%run Loesung/Loesung_5_2.py

## Schritt 6: Evaluation und Visualisierung

### Aufgabe 6.1 

Welche Funktion kann in Keras verwendet werden, um das trainierte Modell zu evaluieren?
Schauen Sie sich dazu die [Dokumentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model) an.

Platz für Notizen

In [ ]:
%load Loesung/Loesung_6_1.py

Die folgende Zelle enthält eine Helferfunktion, die Ihnen das Preprocessing eines einzelnen Bildes abnimmt. Das Bild wird zu einem Array konvertiert. Anschließend wird eine Dimension hinzugefügt, damit das Bild die dimension `(1, width, height, 3)` statt `(width, height, 3)` hat.
Das ist notwendig, da das neuronale Netz am Einagng normalerweise `N` Bilder erwartet also einen Tensor der Dimension `(N, width, height, 3)`. Bei einem einzigen Bild gilt `N=1`. 

In [24]:
from keras.applications.mobilenet import preprocess_input

# helper functions:
def prepare_image(path):
    img = image.load_img(path, target_size=(width, height))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)

### Aufgabe 6.2 

Geben Sie ein paar Bilder mit Ihren vorhergesagten Labels aus. Zu welchem Datensatz gehören die unten abgegebene 4 Beispiele? Schauen Sie sich dazu noch einmal die .csv Dateien an. Warum ist diese Wahl sinnvoll? Ergänzen Sie den folgenden Code an den angegebnen Stellen. 

In [ ]:
from IPython.display import Image, display

# Beispiel-Bilder
img_unsafe = os.path.join("AKIPRO_safety_application_dataset", "unsafe", "akipro_0000002634.jpg")
img_safe_background = os.path.join("AKIPRO_safety_application_dataset", "safe", "akipro_0000000524.jpg")
img_safe_coin_hand = os.path.join("AKIPRO_safety_application_dataset", "safe", "akipro_0000001156.jpg")
img_safe_coin = os.path.join("AKIPRO_safety_application_dataset", "safe", "akipro_0000001666.jpg")

image_paths = [img_unsafe, img_safe_background, img_safe_coin_hand, img_safe_coin]
# Befüllen Sie die Liste mit 0 und 1 gemäß den Beispielbildern
# true_labels = []  # safe -> 0, unsafe -> 1

for (img_path, true_label) in zip(image_paths, true_labels):
    # Ergänzen Sie die folgenden zwei Zeilen
    # img_processed = ...
    # model_prediction = ... 
    predicted_label = np.argmax(model_prediction, axis=1)[0]
    certainty = model_prediction[0][predicted_label]

    display(Image(filename=img_path, width=200))
    print(f"Vorhersage: {predicted_label}, Genauigkeit: {certainty:.6f}, Korrekte Klasse: {true_label}")

In [ ]:
%run Tipps/Tipp_6_2.py

In [ ]:
%load Loesung/Loesung_6_2.py

Was ist hier schief gelaufen? Warum werden die Bilder teilweise mit dem Testdatensatz falsch vorhergesagt? 

Wie bereits festgestellt, gibt es vier Gruppen von Bildern. Eine Gruppe von Bildern wurde vom Netz falsch vorhergesagt, denn sobald Hände zu sehen sind, soll das neuronale Netz das Bild als `unsafe` kategorisieren. In unserem Trainingsdatensatz haben wir absichtlich die Bilder, auf denen Hände und die TÜV-Plakette zu sehen sind, als sicher kategorisiert. 
Dies soll eine Warnung sein, wie einfach es ist, Fehler in selbst erzeugten Datensätzen zu übersehen. Insbesondere für sicherheitskritische Anwendungen ist es wichtig, solche Fehler durch eine geeignete Visualiserung zu erkennen und zu verstehen.

Eine andere Möglichkeit ist, dass das neuronale Netz nur die TÜV-Plakette beachtet und lernt, dass das Bild sicher ist, sobald die Plakette zu sehen ist. Die Hände würden in diesem Fall ignoriert werden.

Um zu erkennen, worauf das neuronale Netz achtet, schauen wir uns nun die Heat Maps, oder auch Attention Maps, von unterschiedlichen Bildern an. Gehen Sie dafür bitte nochmal zurück zu den Videos und schauen Sie sich das nächste Video in diesem Modul an. 

## Schritt 7: Heatmaps erstellen
Versuchen Sie den nachfolgenden Code nachzuvollziehen. Das zugehörige [Paper](https://arxiv.org/abs/1910.01279) kann Ihnen dabei zusätzlich helfen.

In [29]:
from keras import Model
import cv2

def score_CAM(layer_name, preproc_img, model, predicted_label=None):
    # Vorhersage laden, sofern nicht vorhanden
    if predicted_label is None:
        predicted_label = np.argmax(model.predict(preproc_img), axis=1)[0]

    ### Schritt 1: Generieren der intermediate Outputs
    # Erstellen eines Sub-Modells vom ersten Layer bis zum letzen Conv-Layer
    last_conv_layer = model.get_layer(layer_name)
    activation_model = Model(inputs=model.input, outputs=last_conv_layer.output)
    
    # Generieren der Activations
    activations = activation_model.predict(preproc_img)
    # Verschieben der letzten Dimension auf die erste Dimension
    activations = np.transpose(np.squeeze(activations), (2, 0, 1))

    eps = 1e-6

    score_saliency_map = np.zeros((width, height), dtype='float32')
    for activation in activations:
        ### Schritt 2: Upsampling
        # Durch die Pooling Layer sind die Outputs kleiner als das ursprüngliche Bild. Daher ist Upsampling notwendig
        saliency_map = cv2.resize(activation, (height, width), interpolation=cv2.INTER_LINEAR)

        ### Schritt 3: Normalisierung
        # Normalisierung der Attention maps auf den Bereich [0, 1]
        min_map, max_map = np.min(saliency_map), np.max(saliency_map)
        norm_quotient = np.full(shape=(width, height), fill_value=max_map - min_map)
        # eps verhindert eine Division durch 0
        norm_saliency_map = np.divide(saliency_map - min_map, norm_quotient + eps)

        ### Schritt 4: Maskierung
        # Maskierung des Originalbildes
        mask = np.repeat(np.expand_dims(norm_saliency_map, axis=-1), repeats=3, axis=-1)
        masked_img = mask * np.squeeze(preproc_img, axis=0)

        ### Schritt 5: Gewichtung der saliency maps
        score = model.predict(np.expand_dims(masked_img, axis=0))[0][predicted_label]
        score_saliency_map += score * saliency_map

    ### Schritt 6: Normalisierung
    score_saliency_map = np.maximum(0, score_saliency_map)  # RELU
    score_saliency_map_min, score_saliency_map_max = np.min(score_saliency_map), np.max(score_saliency_map)
    score_saliency_map = np.divide(score_saliency_map - score_saliency_map_min, score_saliency_map_max- score_saliency_map_min)

    return score_saliency_map
     

### Aufgabe 7.1
Nun können Sie die reingeladenen Bilder von oben wieder verwenden, um herauszufinden, worauf das Netz achtet, wenn das Bild ihm präsentiert wird. 

Mit welcher Funktion werden die Heat Maps/Attention Maps generiert? Welchen Input hat diese Funktion? Und was ist ihr Output?

In [ ]:
# Laden Sie die Bilder aus der vorherigen Aufgabe
# attention_map_safe_background = ...

In [31]:
%load Loesung/Loesung_7_1.py

### Aufgabe 7.2 Visualiserung
Ergänzen Sie den Code am Ende der Zelle um die Heatmaps der 4 Bilder anzuzeigen. Nutzen sie dazu die Funktion `plot_heatmap` und beachten sie das Argument `plot_idx`(startet bei 1 und muss pro Bild "hochgezählt werden").

In [ ]:
def plot_heatmap(fig, img, heatmap, model, title, plot_idx, rows=2, cols=2):
    logit = model.predict(img)
    label = np.argmax(logit, axis=1).item()
    x_label = f"label: {label},  mit Sicherheit: {logit[0][label]:.6f}"

    fig.add_subplot(rows, columns, plot_idx)
    plt.title(title)
    plt.imshow(img[0,:,:, :])
    plt.imshow(heatmap, alpha=0.5)
    plt.xlabel(x_label)
    plt.xticks([])
    plt.yticks([])

# Bilder laden
img_safe_background_processed = prepare_image(img_safe_background)
img_safe_coin_hand_processed = prepare_image(img_safe_coin_hand)
img_safe_coin_processed = prepare_image(img_safe_coin)
img_unsafe_processed = prepare_image(img_unsafe)

# Plot Einstellungen
rows, columns = 2, 2
fig = plt.figure(figsize=(15, 10))
fig.suptitle('Attention Maps')

# Bilder mit Heatmaps plotten
# Platz für Ihren Code
# 
# 
#
plt.show()


In [ ]:
%load Loesung/Loesung_7_2.py


### Aufgabe 7.3

Wie lassen sich die obigen Heat Maps interpretieren?

In [ ]:
%run Loesung/Loesung_7_3.py